# Databricks: platform for DevOps

Lately I have been practicing spark, python,  SQL, and R on the community Databricks platform (https://community.cloud.databricks.com). Databricks has several kernels with a simple jupyter-notebook cell interface, such that you can manipulate large quantities of data to quickly exploite the best features of different programming languages. For example, if you need to manipulate a large database you can preprocess the data quickly using SQL, then save the necessary data to a 'fast retrival' table called a Delta Table. Then you can do more complicated manipulations with the 'fast table', like machine learning or deep learning modeling in python AND/OR statistical analysis in R.

<img src="main_splash.png" alt="Drawing" style="width: 500px;"/>

Some basic commands to get started on this platform are: 0) loading data, 1) saving data to a Delta Table, 2) standard pre-processing in SQL, 3) conversion of SQL table to python and R.
Databricks even has an option for publishing the notebook online: https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/6791636491620955/2798567329490024/4779345166745182/latest.html. The published notebooks are only available for 6 month, so I also put the notebook on Git. Below are highlights from the notbook. Enjoy.

## Loading data

### Load from csv: create a pyspark Dataframe

In [ ]:
%python
# Example 1: File location and type
file_location = "/FileStore/tables/bikeshare-1.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "True"  #"false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_pyspark = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_pyspark)

In [ ]:
%python
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType, IntegerType

# Example 2: Below, I add onto the command and perform an initial pre-processing 
# of data directly with the loading command.
file_location = "/mnt/training/healthcare/tracker/moocs/daily-metrics.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "True"  #"false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_pyspark = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location) \
  .select(
        "device_id",
        "dte",
        col("resting_heartrate").cast(DoubleType()),
        col("active_heartrate").cast(DoubleType()),
        col("bmi").cast(DoubleType()),
        col("vo2").cast(DoubleType()),
        col("workout_minutes").cast(DoubleType()),
        "lifestyle",
        col("steps").cast(IntegerType())
      )
display(df_pyspark)

### Save pyspark Dataframe As Delta Table

In [ ]:
%python
df_pyspark.write.format("delta").mode("overwrite").save("/FileStore/tables/ht_daily_metrics")

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS delta_ht_daily_metrics
USING DELTA LOCATION '/FileStore/tables/ht_daily_metrics'

### Read pyspark Delta Table in SQL

In [ ]:
%sql
SELECT * FROM delta_ht_daily_metrics;

### My standard preprocessing Framework for small SQL Dataframes

I find that strategies for making an SQL query are rarely discussed. It can sometimes be troublesome to manipulate scaler information with DataFrame columns, even with the "WITH" command and making Temporary Tables. Below, I developed a standardized organization for adding scalar information, like the max, min, etc, directly into the table such that all needed information for calculating simple equations like t-statistic/z-score are possible.

In step 1, I make a column of zeros that will serve as a way to store scalar values. In step 2, I save all scalar values to a column in the DataFrame. And, then in step 3, I add comparison scalar column values information to the DataFrame. 

Adding repeating column information to a DataFrame does seem computaionally wasteful, but for small datasets this seems like the most convient method of manipulation both column and scalar information. There are vector manipulation functions in SQL like TRANSFORM and FILTER, so maybe an alternative to this framework could be to save vectors and scalars with the Dataframe.

In [ ]:
%sql
-- Create a DataFrame of columns with both scalar and column information for Analysis
-- How to preprocess in SQL :[0] always add certain manipulation columns (row_num, zero_column to add scalar values to), [1] add certain manipulation scalars (min/max/frequency count of some variables).

CREATE OR REPLACE TEMPORARY VIEW ht_daily_metrics2 AS
WITH temptab AS
(
-- Step 0: Store column transformations HERE
SELECT *, 
CAST(workout_minutes AS INT) AS wm_int,
IF(WEEKDAY(dte) > 4, 2, 5) AS normval,
IF(WEEKDAY(dte) > 4, "weekend", "weekday") AS wday,
CASE WHEN lifestyle = 'Cardio Enthusiast' THEN 1 WHEN lifestyle = 'Athlete' THEN 2 WHEN lifestyle = 'Weight Trainer' THEN 3 WHEN lifestyle = 'Sedentary' THEN 4 END AS lifestyle_num,
ROW_NUMBER() OVER(ORDER BY device_id) AS num_row FROM delta_ht_daily_metrics
)
-- Step 1: Create zero_col to do scalar-column transformations
SELECT *, (num_row*0) AS zero_col FROM temptab;


-- Step 2: Store scalar and existing column transformations HERE
CREATE OR REPLACE TEMPORARY VIEW ht_daily_metrics3 AS
WITH temptab AS
(
-- Store scalars into columns HERE
SELECT *,
zero_col+(SELECT AVG(resting_heartrate) FROM ht_daily_metrics2) AS samp1_mean,
zero_col+(SELECT AVG(active_heartrate) FROM ht_daily_metrics2) AS samp2_mean,
zero_col+(SELECT STD(resting_heartrate) FROM ht_daily_metrics2) AS samp1_std,
zero_col+(SELECT STD(active_heartrate) FROM ht_daily_metrics2) AS samp2_std,
zero_col+(SELECT COUNT(*) FROM ht_daily_metrics2) AS samp1_2_len
FROM ht_daily_metrics2
)
-- Step 3: Store comparisons of scalars columns HERE
SELECT *,
zero_col+(SELECT IF(MIN(samp1_mean) > MIN(samp2_mean), MIN(samp2_mean), MIN(samp1_mean)) FROM temptab) AS mean_comp_min
FROM temptab;


SELECT * FROM ht_daily_metrics3;

### Create SQL functions for t-statistic/z-score

In [ ]:
%sql
CREATE OR REPLACE FUNCTION t_OR_Z_statistic_twosample( samp1 FLOAT, samp2 FLOAT, samp1_mean FLOAT, samp2_mean FLOAT, samp1_std FLOAT, samp2_std FLOAT, samp1_len FLOAT, samp2_len FLOAT)
RETURNS FLOAT
RETURN ((samp1 - samp2) - (samp1_mean - samp2_mean))/SQRT( ((samp1_std*samp1_std)/samp1_len) + ((samp2_std*samp2_std)/samp2_len));

In [ ]:
%sql
-- Calculate the pdf (the normal distribution OR the probability density function)
CREATE OR REPLACE FUNCTION normal_pdf( t_OR_Z FLOAT, t_OR_Z_mean FLOAT, t_OR_Z_std FLOAT)
RETURNS FLOAT
RETURN (1/(SQRT(2*PI())*t_OR_Z_std))*EXP(-((t_OR_Z - t_OR_Z_mean)*(t_OR_Z - t_OR_Z_mean))/(2*t_OR_Z_std*t_OR_Z_std));

In [ ]:
%sql
CREATE OR REPLACE TEMPORARY VIEW ht_daily_metrics5 AS
WITH temp_tab AS
(
SELECT *,
zero_col+(SELECT AVG(t_OR_Z_statistic_2samp) FROM ht_daily_metrics4) AS t_OR_Z_statistic_2samp_mean,
zero_col+(SELECT STD(t_OR_Z_statistic_2samp) FROM ht_daily_metrics4) AS t_OR_Z_statistic_2samp_std
FROM ht_daily_metrics4
)
SELECT *,
normal_pdf( t_OR_Z_statistic_2samp, t_OR_Z_statistic_2samp_mean, t_OR_Z_statistic_2samp_std) AS t_OR_Z_statistic_2samp_pdf
FROM temp_tab;

SELECT * FROM ht_daily_metrics5;

In [ ]:
-- Run the function in a query just below
SELECT t_OR_Z_critical_twosample(samp1_mean, samp2_mean, samp1_std, samp2_std, samp1_2_len, samp1_2_len) AS t_OR_Z_statistic_2samp FROM ht_daily_metrics5
LIMIT 1;

### Convert SQL Table to pandas DataFrame

In [ ]:
%python
df_pandas = spark.sql("SELECT * FROM ht_daily_metrics3").toPandas()
df_pandas.head()

In [ ]:
%python
from scipy.stats import ttest_ind
resting_hr = spark.sql("SELECT resting_heartrate FROM ht_daily_metrics3").toPandas()
active_hr = spark.sql("SELECT active_heartrate FROM ht_daily_metrics3").toPandas()

ttest_ind(resting_hr, active_hr, equal_var = False)

### Convert pandas DataFrame to numpy Array

In [ ]:
%python
df_numpy = df_pandas.to_numpy()
df_numpy

### Load SQL Table in R

In [ ]:
%r
library(SparkR)
library(sparklyr)
library(dplyr)

In [ ]:
%r
# After you load sparklyr, you must call sparklyr::spark_connect to connect to the cluster, specifying the databricks connection method.
sc <- spark_connect(method = "databricks")

In [ ]:
%r
R_ht_daily_metrics3 <- collect(sdf_sql(sc, "SELECT * FROM ht_daily_metrics3"))

In [ ]:
%r
print(R_ht_daily_metrics3)

In [ ]:
%r
# Selecting columns in a DataFrame requires library(dplyr)
a_hr <- R_ht_daily_metrics3 %>% select(active_heartrate)
r_hr <- R_ht_daily_metrics3 %>% select(resting_heartrate)

In [ ]:
%r
install.packages("ggpubr")

In [ ]:
%r
t.test(a_hr, r_hr, alternative = "two.sided", var.equal = FALSE)